# **Mini-Project I**
For this project I practiced handling of complex lists and dictionaries in Python. Also, by answering very specific transportation questions, learned how to navigate through an API documentation to make the correct resquets [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).


In [1]:
import requests
import json
import os

## **Grabbing foursquare api key and id from bash profile**

In [2]:
# don't forget package os
app_id = os.environ["app_id"]
app_key = os.environ["app_key"]
url_append = f'?app_id={app_id}&app_key={app_key}' 

In [3]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"
print(url+url_append)

https://api.tfl.gov.uk/AirQuality?app_id=e0938d95391d465fb9be5ae179be2e5c&app_key=aae04a1ef67b491fa360b1c107e5cda1


In [4]:
# Sent send the request to the API using formatted string containing the imported api key and id
res = requests.get(url+url_append)

In [5]:
# We can check if the request was successful (http protocol of 200 refers to a successful api request)
res.status_code

200

In [6]:
# We can export the information that was returned using method .json()
air_json = res.json()

In [8]:
#print(json.dumps(air_json, indent=2))

## **Task 1**
### Determining the AirQuality predictions for the coming day

In [9]:
#checking for the current forecast key in the air quality JSON, appending the result to some empty list
summary = []
for i in air_json["currentForecast"]:
    summary.append(i["forecastSummary"])
    break

In [10]:
print(summary)

['Low air pollution forecast valid from Tuesday 8 June to end of Wednesday 9 June GMT']


## **Task 2**
### Finding out different modes of  transport which are operated by Transfer for London. Must determine not only how many of modes do they have but check whether the modes are still in service


In [13]:
#using the Journey documentation section, found the meta modes section was where the number of transport modes in London could be found
url_2 = "https://api.tfl.gov.uk/Journey/meta/modes"
re2 = requests.get(url_2)

transportation_json = re2.json()

In [160]:
#Must check the condition that the avalaible mode is still running (the boolean value of the key isTflService), and then can append said methods into a empty list
transportation_list = []
for i in transportation_json:
    if i['isTflService'] == True:
        transportation_list.append(i['modeName'])

print(transportation_list)
"There exists {} modes of transpotation in London".format(len(transportation_list))

['bus', 'cable-car', 'cycle-hire', 'dlr', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'tflrail', 'tram', 'tube']


'There exists 11 modes of transpotation in London'

## **Task 3**
### How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [11]:
#no special parameters required for this http requests URL
re3 = requests.get('https://api.tfl.gov.uk/BikePoint')
bike_json = re3.json()

#bike_json, commented out as api set back an extremely long JSON

In [150]:
"The number of bike points operated in London: {}".format(len(bike_json))

'The number of bike points operated in London: 788'

## **Task 4**
### Figuring out how many tube and bus lines are in London ###

In [114]:
#two seperate htttp requests, one for bus and tube. Both from the line "module" of TfL
re4 = requests.get('https://api.tfl.gov.uk/Line/Mode/bus')
bus_json = re4.json()


re5 = requests.get('https://api.tfl.gov.uk/Line/Mode/tube')
tubes_json = re5.json()

In [115]:
#iterated through the bus json file referencing all values with the key of 'name' due to the fact that we were asked to get the names of all London bus names, found bus names were characters combined with integers so decided to just return 
#number of bus' present  
amount_of_buses = len(bus_json)
busNames = []

for i in bus_json:
    busNames.append(i['name'])

print(len(set(busNames))) #ensured only unique bus names are printed by explicity casting the list to be a set

679


In [ ]:
tubes_json

In [155]:
#iterated through the tubes json file refencing all values with the key of 'name' due to the fact that we were asked to get the names of all London tubes
amount_of_tubes = len(tubes_json)

for j in tubes_json:
    print (j['name'])#returned the number of tubes as they are all nice names compared difficult to interpret numbers
    

Bakerloo
Central
Circle
District
Hammersmith & City
Jubilee
Metropolitan
Northern
Piccadilly
Victoria
Waterloo & City


## **Task 5**
### For the tube `victoria` line, finding out how many stations exist ###

In [156]:
re5 = requests.get('https://api.tfl.gov.uk/Line/Victoria/StopPoints')
victoria_json = re5.json()
#print(json.dumps(victoria_json, indent=2))

In [157]:
print(len(victoria_json))

16


## **Task 6**
### Planned a journey from Heathrow Airport to Tower Bridge via Bus and Tube, utilizing the api to figure out which method is faster?###


In [127]:
re6 = requests.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.4700%2C20-0.4543/to/51.5055%2C20-0.0754?mode=tube')
re7 = requests.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.4700%2C20-0.4543/to/51.5055%2C20-0.0754?mode=bus')

In [129]:
tube_json = re6.json()
bus_json = re7.json()

In [148]:
time_difference = 0
if (re6.status_code & re7.status_code) == 200:
    bus_journey = bus_json['journeys']
    tube_journey = tube_json['journeys']
    bus_time = [] 
    tube_time = []
    
    for i in bus_journey:
        bus_time.append(i.get('duration'))
    
    for j in tube_journey:
        tube_time.append(j.get('duration'))
        

    if min(bus_time) < min(tube_time):
        time_difference = min(tube_time) - min(bus_time)
        print("The Bus had a quicker route to Heathrow Aiport by a difference of {} minutes".format(time_difference))
    
    elif min(tube_time) < min(bus_time):
        time_difference = min(bus_time) - min(tube_time)
        print("The Tube had a quicker route to Heathrow Aiport by a difference of {} minutes".format(time_difference))
    
    else:
        print("Both methods are equally fast!")
              
print(bus_time)
print(tube_time)


The Tube had a quicker route to Heathrow Aiport by a difference of 16 minutes
[107, 99, 102]
[320, 86, 83]
